In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
train_dir = '../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train'
test_dir = '../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'
print(os.listdir('../input/asl-alphabet/asl_alphabet_test/asl_alphabet_test'))
print(test_dir)

In [ ]:
import os
import cv2
labels_dict={'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25,'space':26,'del':27,'nothing':28}
labels=[]
image_data=[]
size=64,64
print("[INFO]Loading Data...")
cnt=0
for folder in os.listdir(train_dir):
    for file in os.listdir(train_dir+'/'+folder):
        filepath=train_dir+'/'+folder+'/'+file
        image=cv2.imread(filepath)
        final_img=cv2.resize(image,size)
        #final_img=cv2.cvtColor(final_img,cv2.COLOR_BGR2RGB)
        image_data.append(final_img)
        labels.append(labels_dict[folder])
        cnt=cnt+1
        print(cnt)
        



In [ ]:
#print(labels)
print(image_data[0])

In [ ]:
import matplotlib.pyplot as plt
for i in range(0,10):
    for j in range(0,1):
        plt.imshow(image_data[i])
        plt.show()
        break

In [ ]:
import numpy as np
image_data=np.array(image_data)
imgae_data=image_data.astype('float32')/255.0 #normalization

In [ ]:
import keras
labels=keras.utils.to_categorical(labels)
print(labels)

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_val,Y_train,Y_val=train_test_split(image_data,labels,test_size=0.05)


In [ ]:
print(len(X_train))
print(len(X_val))

In [ ]:
#Design of model
from keras.models import Sequential
from keras import regularizers
from keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout,BatchNormalization
model = Sequential()
model.add(Conv2D(16, kernel_size = [3,3], padding = 'same', activation = 'relu', input_shape = (64,64,3)))
model.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = [3,3]))
model.add(Conv2D(32, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(Conv2D(64, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = [3,3]))
model.add(Conv2D(128, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(Conv2D(256, kernel_size = [3,3], padding = 'same', activation = 'relu'))
model.add(MaxPool2D(pool_size = [3,3]))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(512, activation = 'relu', kernel_regularizer = regularizers.l2(0.001)))
model.add(Dense(29, activation = 'softmax'))
model.compile(optimizer = 'adam', loss = keras.losses.categorical_crossentropy, metrics = ["accuracy"])    
print("MODEL CREATED")
model.summary()

In [ ]:
model_fit=model.fit(X_train,Y_train,batch_size=64,epochs=10,validation_split=0.1)

In [ ]:
model.save('fix_my_model_1.h5')

In [ ]:
from keras.models import load_model
new_model=load_model('fix_my_model_1.h5')
score = new_model.evaluate(x = X_val, y = Y_val, verbose = 0)
print('Accuracy for test images:', round(score[1]*100, 3), '%')
#score = new_model.evaluate(x = X_eval, y = y_eval, verbose = 0)
#print('Accuracy for evaluation images:', round(score[1]*100, 3), '%'

In [ ]:
names=[]
test_image_data=[]
size=64,64
print("[INFO]Loading Data...")
cnt=0
for image in os.listdir(test_dir):
    filepath=test_dir+'/'+image
    img=cv2.imread(filepath)
    final_img=cv2.resize(img,size)
    #final_img=cv2.cvtColor(final_img,cv2.COLOR_BGR2RGB)
    test_image_data.append(final_img)
    names.append(image)
    cnt=cnt+1
    print(cnt)

In [ ]:
predictions = [new_model.predict_classes(image.reshape(1,64,64,3))[0] for image in test_image_data]

In [ ]:
prediction_labels=[]
for i in range(len(predictions)):
    for key in labels_dict:
        if predictions[i]==labels_dict[key]:
            prediction_labels.append(key)
            break
        
print(prediction_labels)
print(names)